## Introdução

Os dados `cps-earnings` incluem dados de rendimentos para 2014, obtidos dos conjuntos de dados _Merged Outgoing Rotation Groups_ (MORG) do _Current Population Survey_ (CPS) dos EUA. O conjunto de dados tem N=149316 observações.

Variáveis-chave: diferença salarial de mulheres e homens entre analistas de mercado, remuneração por hora e idade dos analistas de mercado, idade e gênero dos funcionários com pós-graduação, três categorias de pós-graduação (mestrado, profissionalizante e doutorado), entre outras.

# Códigos em Python

Como sempre, começamos importando as bibliotecas e funções necessárias para a análise.

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu, wilcoxon, kruskal

Os dados são novamente carregados a partir da URL fornecida pelo website do livro-texto.

In [ ]:
cps = pd.read_csv("https://osf.io/4ay9x/download/")

Sempre que carregamos dados novos para trabalhar, devemos verificar a documentação destes dados. Esta documentação deve trazer a fonte primária dos mesmos, assim como o significado de cada uma das variáveis que compõe o dataset. No presente caso temos um arquivo de Readme e também o dicionário de dados, com os links abaixo:

[Readme.txt](https://osf.io/cgfrq)

[Dicionário de dados](https://osf.io/uqe8z)

Estamos interessados em fazer testes de comparação de localização central de forma **não paramétrica**. Uma das necessidades para tais testes é a ausência de hipótese a priori da distribuição da população e uma **quantidade pequena de observações**.

Para tanto, vamos testar os **ganhos semanais** de CEOs do Estado da Flórida que possuem apenas graduação. Somos assim específicos para filtrarmos a base de dados e obter um pequeno número de observações. 

In [ ]:
# Estado da Flórida, graduação completa (Bachelors degree), CEOs (chief executives)
filter = (cps["stfips"] == "FL") & (cps["grade92"] == 43) &(cps["occ2012"] == 10)
df = cps.loc[filter]
df.info()

## Teste de Wilcoxon 

Hipótese nula, $H_0: Md = Md_0$

Hipótese alternativa $H_a: Md\neq Md_0$

Usamos com amostras menores que 30 observações 

Estatística de teste: $T=min\{T^-, T^+\}$

Regra de decisão: Se $T\leq T_{tab}$, então rejeita-se a $H_0$

Em nosso exemplo vamos testar se a **mediana** dos ganhos semanais de CEOs é igual a US\$ 2.000.

A função `wilcoxon` faz um teste entre amostras pareadas, que não é nosso caso aqui. Entretanto, podemos entender o teste de uma única amostra contra um valor fixo como uma amostra pareada onde a segunda amostra possui sempre o mesmo valor. Passamos para a função a diferença entre as amostras pareadas.

In [ ]:
df["earnwke"].median()

In [ ]:
wilcoxon(df["earnwke"] - 2000)

## Teste de Mann-Whitney

Hipótese nula, $H_0: X_1 \text{ e } X_2$ possuem mesma distribuição

Hipótese alternativa $H_a: X_1 \text{ e } X_2$ diferem em sua posição central

Usamos com amostras menores que 20 observações em cada grupo

Estatística de teste: $U=min\{U_1, U_2\}$

Regra de decisão: Se $U\leq U_{tab}$, então rejeita-se a $H_0$

Faremos agora um teste de comparação entre duas amostras, portanto, necessitamos de um outro Estado para comparar. Escolhemos o Texas.

In [ ]:
# Estado do Texas, graduação completa (Bachelors degree), CEOs (chief executives)
filter2 = (cps["stfips"] == "TX") & (cps["grade92"] == 43) &(cps["occ2012"] == 10)
df2 = cps.loc[filter2]
df2.info()

In [ ]:
df2["earnwke"].median()

In [ ]:
mannwhitneyu(df["earnwke"], df2["earnwke"])

## Teste de Kruskal-Wallis

Hipótese nula, $H_0$: todos os $k$ grupos possuem mesma distribuição

Hipótese alternativa $H_a$: pelo menos um dos grupos difere em sua posição central

Usamos com amostras maiores que 4 observações em cada grupo

Estatística de teste: $H=\frac{12}{N(N+1)}\sum_{i=1}^k \frac{T_i^2}{n_i} - 3(N+1)$

Regra de decisão: Se $H\geq \chi^2_{k-1,\alpha}$, então rejeita-se a $H_0$

Finalmente, no teste de Kruskal-Wallis a comparação é feita entre 3 ou mais amostras. Escolhemos o Tenessee como o terceiro estado a ser testado. Verifiquem que todas as 3 amostras possuem poucas observações ($n \leq 30$).

In [ ]:
# Estado do Tenessee, graduação completa (Bachelors degree), CEOs (chief executives)
filter3 = (cps["stfips"] == "TN") & (cps["grade92"] == 43) &(cps["occ2012"] == 10)
df3 = cps.loc[filter3]
df3.info()

In [ ]:
df3["earnwke"].median()

In [ ]:
kruskal(df["earnwke"], df2["earnwke"], df3["earnwke"])